# 优化思路和详细解释

## 导入必要的库
- 引入了多种库，包括用于数值计算的NumPy，深度学习和数值计算的Theano，贝叶斯统计建模的PyMC3，以及数据处理的Pandas等。这些库为实验设计提供了强大的功能支持。
- 使用主成分分析（PCA）进行特征降维，确保在处理高维数据时保留主要信息。数据标准化有助于提高模型的收敛性和稳定性。
- 高斯过程回归（GPR）用于处理不确定性估计和预测性能。通过交叉验证方法评估模型性能，避免过拟合和欠拟合问题。

## 重新分桶和设计实验
### 特征筛选和预处理
- 定义`preprocess_features`函数，使用PCA对特征进行降维，保留主要信息，确保降维后的数据仍然具有代表性。
- 定义`redefine_space`函数，使用StandardScaler对特征进行标准化处理，确保所有特征的尺度一致。
- 定义`bucketize_space`函数，根据特征分布将团队空间划分为若干子空间，确保每个子空间内的数据点具有相似性，便于捕捉细微差异。

### 模型构建和评估
- 在`build_model`函数中，使用高斯过程来表示每个系数，定义了高斯过程的先验分布，并假设观测值带有高斯噪声。通过定义合适的模型和噪声先验，确保模型能够准确反映数据的不确定性。
- 使用MCMC进行采样，获取参数的后验分布，并定义性能度量函数，计算总体性能。

### 采样策略优化
- 定义了`select_max_variance`函数，作为采集函数，优先选择不确定性最大或预期性能改进最大的采样点。通过最大化方差的策略，确保在有限的实验次数内获取最大的信息量。
- 在每次采样后，根据新获取的数据，实时更新模型和采集函数，动态调整采样策略，确保实验设计的灵活性和适应性。

### 模拟和数学证明
- 通过模拟和数学证明，评估策略的最优性，确保在科学上避免泛化问题。使用蒙特卡罗模拟方法，评估不同采样策略的效果，选择表现最优的策略应用于实际实验中。

### 实验执行和结果分析
- 在实际实验中执行优化后的采样策略，并对结果进行详细分析。逐步执行实验，实时记录和分析每次采样结果，确保实验结果的准确性和有效性。
- 对实验结果进行详细分析，总结不同特征和采样策略对实验质量的影响，识别关键因素和优化点。

In [ ]:
# 导入必要的库
import numpy as np  # 数值计算
import theano  # 深度学习和数值计算
import pymc3 as pm  # 贝叶斯统计建模
import arviz as az  # 用于贝叶斯分析结果可视化
import pandas as pd  # 数据处理
from scipy.stats import ttest_ind  # 统计检验
import matplotlib.pyplot as plt  # 绘图
import matplotlib as mpl  # 绘图设置
from sklearn.decomposition import PCA  # 主成分分析
from sklearn.preprocessing import StandardScaler  # 数据标准化
from sklearn.gaussian_process import GaussianProcessRegressor  # 高斯过程回归
from sklearn.model_selection import cross_val_score  # 交叉验证
from sklearn.multioutput import MultiOutputRegressor  # 多输出回归

# 设置绘图的默认字体和大小
mpl.rcParams['font.sans-serif'] = ['KaiTi', 'SimHei', 'FangSong']
mpl.rcParams['font.size'] = 12
mpl.rcParams['axes.unicode_minus'] = False

# 更新绘图参数，隐藏一些不必要的轴
plt.rcParams.update({
    'axes.spines.bottom': True,
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False
})


In [ ]:
# 定义“真实值”函数
def measure_task(t):
    # 定义任务空间中任务位置 t 对应的系数
    Cx = t + 1
    Cy = -t + 1
    b = 0
    return Cx, Cy, b

# 生成任务空间的取样点
tt = np.linspace(-1, 1, 100)
Cx, Cy, b = measure_task(tt)

# 绘制任务空间中系数随任务位置变化的图
plt.figure(figsize=(8, 3))
plt.plot(tt, Cx, label="Cx")
plt.plot(tt, Cy, label="Cy")
plt.xlabel("任务空间中的一维位置 (t)")
plt.legend()
plt.title("图1：任务空间中的系数")


In [ ]:
# 定义世界模型和测量函数
def ground_truth(x, y, t):
    # 在团队空间点(x, y)和任务空间点't'处计算“真实”输出值z
    Cx_t, Cy_t, b = measure_task(t)
    z = Cx_t * x + Cy_t * y + b
    return z

def measure(x, y, t):
    # 在团队空间点(x, y)和任务空间点't'处进行一次实验，并返回带有测量噪声的结果
    sigma = .5  # 设置噪声标准差
    z_measured = ground_truth(x, y, t) + np.random.normal(loc=0, scale=sigma, size=np.shape(x))
    return z_measured

# 测量在(x=1, y=1, t=1)处的结果
measure(x=1, y=1, t=1)


In [ ]:
# 绘制团队空间中的真实值
xx, yy = np.meshgrid(np.linspace(-1, 1, 50), np.linspace(-1, 1, 50))

plt.figure(figsize=(10, 4))

# 在t=-0.25处绘制团队空间的真实值
plt.subplot(1, 2, 1)
t = -.25
zz_observations = measure(xx, yy, t)
plt.contourf(xx, yy, zz_observations)
plt.xlabel("因子 $x$")
plt.ylabel("因子 $y$")
plt.colorbar(label="性能 $z$")
plt.title('"真实值"在t=%.02f' % t)

# 在t=1处绘制团队空间的真实值
plt.subplot(1, 2, 2)
t = 1
zz_observations = measure(xx, yy, t)
plt.contourf(xx, yy, zz_observations)
plt.xlabel("因子 $x$")
plt.ylabel("因子 $y$")
plt.colorbar(label="性能 $z$")
plt.title('"真实值"在t=%.02f' % t)

plt.suptitle("图2：任务空间不同位置下的团队空间")
plt.tight_layout()


In [ ]:
# 重新分桶和设计实验

# 特征筛选和预处理
def preprocess_features(features, n_components):
    # 使用PCA进行降维，保留主要信息
    pca = PCA(n_components=n_components)
    return pca.fit_transform(features)

# 重新定义团队空间和任务空间
def redefine_space(features):
    # 标准化特征
    scaler = StandardScaler()
    return scaler.fit_transform(features)

# 分桶策略
def bucketize_space(features, n_buckets):
    # 将特征值进行分桶处理
    buckets = np.linspace(features.min(), features.max(), n_buckets)
    bucketed_features = np.digitize(features, buckets)
    return bucketed_features


In [ ]:
# 构建MCMC模型
def build_model(x_sampled, y_sampled, t_sampled, z_observations, gp_length_scale=.4):
    n_samples = x_sampled.shape
    with pm.Model() as taskspace_hierarchical_model:
        # 定义高斯过程来表示每个系数
        cov_func = pm.gp.cov.ExpQuad(1, ls=gp_length_scale)
        gpx = pm.gp.Latent(cov_func=cov_func)
        gpy = pm.gp.Latent(cov_func=cov_func)

        # 在任务空间中为每个系数放置高斯过程先验
        Cx = gpx.prior("Cx", X=t_sampled[:, None], shape=n_samples)
        Cy = gpy.prior("Cy", X=t_sampled[:, None], shape=n_samples)

        # 截距的正态分布先验
        b = pm.Normal("b", mu=0, sd=1)
        # 噪声的伽马分布先验
        noise = pm.Gamma("noise", alpha=2, beta=1)

        # 定义团队空间中的线性函数形式
        z = pm.Deterministic("z", Cx * x_sampled + Cy * y_sampled + b)
        # 假设观测值带有高斯噪声
        z_measured = pm.Normal("z_measured", mu=z, sd=noise, observed=z_observations)
        
    return taskspace_hierarchical_model, (gpx, gpy, b)

n_samples = 60
# 生成随机样本
xs, ys, ts = np.random.uniform(-1, 1, size=(3, n_samples))
z_observations = measure(xs, ys, ts)

# 构建模型
taskspace_hierarchical_model, elements = build_model(xs, ys, ts, z_observations)
# 绘制模型结构图
pm.model_to_graphviz(taskspace_hierarchical_model)


In [ ]:
# 进行MCMC采样
with taskspace_hierarchical_model:
    trace = pm.sample(1000)


In [ ]:
# 定义性能度量函数
def draw_task_space(taskspace_hierarchical_model, elements, trace, ax, plot_t=np.linspace(-1, 1, 50)):
    gpx, gpy, b = elements
    
    with taskspace_hierarchical_model:
        # 在任务空间上定义后验预测
        Cx_plot = gpx.conditional("Cx_plot", plot_t[:, None])
        Cy_plot = gpy.conditional("Cy_plot", plot_t[:, None])
        C_functions = pm.sample_posterior_predictive(trace, var_names=["Cx_plot", "Cy_plot"], samples=500)
        
    # 绘制Cx的高斯过程分布
    plot_gp_dist(ax, C_functions["Cx_plot"], plot_t, palette="Blues", fill_alpha=.05, samples_alpha=.05)
    ax.plot([], [], 'cornflowerblue', alpha=1, linewidth=2, label="Cx")
    
    # 绘制Cy的高斯过程分布
    plot_gp_dist(ax, C_functions["Cy_plot"], plot_t, palette="Oranges", fill_alpha=.05, samples_alpha=.05)
    ax.plot([], [], 'darkorange', alpha=1, linewidth=2, label="Cy")
    ax.set_ylabel("系数值")
    
    # 计算所有点的绝对t统计量
    t_statistics = np.array([np.abs(ttest_ind(C_functions['Cx_plot'][:, i], C_functions['Cy_plot'][:, i]).statistic) for i in range(len(plot_t))])
    ax2 = ax.twinx()
    ax2.plot(plot_t, t_statistics, 'dimgrey', linewidth=2)
    ax2.set_ylabel("T统计量")
    ax.plot([], [], 'dimgrey', linewidth=2, label="T统计量")
    
    # 计算总体性能
    performance = 1/np.mean(1/t_statistics)
    ax.set_title("总体性能: %.02f" % performance)
    
    plt.xlabel("任务空间中的一维位置")
    return C_functions

plt.figure(figsize=(8, 3))
ax = plt.subplot(1, 1, 1)
C_functions = draw_task_space(taskspace_hierarchical_model, elements, trace, ax=ax)


In [ ]:
# 定义新的采集函数
def select_max_variance(taskspace_hierarchical_model, trace, candidate_xs, candidate_ys, candidate_ts, elements):
    gpx, gpy, b = elements
    with taskspace_hierarchical_model:
        # 在任务空间上定义后验预测
        Cx_candidates = gpx.conditional("Cx_candidates", candidate_ts[:, None])
        Cy_candidates = gpy.conditional("Cy_candidates", candidate_ts[:, None])
        # 给定任务空间中位置的系数样本，预测团队空间中的性能
        performance_pred = pm.Deterministic("performance_pred", Cx_candidates * candidate_xs + Cy_candidates * candidate_ys + b)
        predictions = pm.sample_posterior_predictive(trace, var_names=["performance_pred"], samples=500)
        
    # 计算每个候选样本点的得分（方差）
    score = predictions['performance_pred'].var(axis=0)
    # 找到方差最大的候选点的索引
    choice_index = np.argmax(score)
    
    return candidate_xs[choice_index], candidate_ys[choice_index], candidate_ts[choice_index]

n_candidates = 3
candidate_xs, candidate_ys, candidate_ts = np.random.uniform(-1, 1, size=(3, n_candidates))
print("候选点:")
print('\n'.join([str(a) for a in zip(candidate_xs, candidate_ys, candidate_ts)]))

selection = select_max_variance(taskspace_hierarchical_model, trace, candidate_xs, candidate_ys, candidate_ts, elements)
print("选定点:")
selection


In [ ]:
# 任务空间上的主动学习
n_initial_samples = 2
xs, ys, ts = np.random.uniform(-1, 1, size=(3, n_initial_samples))
z_observations = measure(xs, ys, ts)

candidate_xs, candidate_ys, candidate_ts = np.meshgrid(
    np.linspace(-1, 1, 2),
    np.linspace(-1, 1, 2),
    np.linspace(-1, 1, 20),
)

n_iterations = 30
for i in range(n_iterations):
    # 拟合替代模型
    taskspace_hierarchical_model, elements = build_model(xs, ys, ts, z_observations)
    with taskspace_hierarchical_model:
        trace = pm.sample(1000)

    # 预测候选点的结果并选择最佳点
    selection = select_max_variance(
        taskspace_hierarchical_model, trace, 
        candidate_xs.flatten(), candidate_ys.flatten(), candidate_ts.flatten(), 
        elements
    )
    
    # 进行下一个样本的测量
    next_observation = measure(*selection)
        
    # 绘制当前知识状态和下一个决策
    plt.figure(figsize=(8, 3))
    ax = plt.subplot(1, 1, 1)
    C_functions = draw_task_space(taskspace_hierarchical_model, elements, trace, ax=ax)
    ax.vlines(ts, *ax.get_ylim(), color='grey', label="之前的选择", alpha=.4)
    ax.vlines(selection[2], *ax.get_ylim(), color='k', label='选择的任务', alpha=.8)
    plt.tight_layout()
    plt.annotate("%i 个观测值" % len(z_observations), xy=(.1, .8), xycoords='axes fraction', fontsize=14)
    ax.legend(loc="upper right")
    
    # 将新样本添加到数据集中
    xs = np.append(xs, selection[0])
    ys = np.append(ys, selection[1])
    ts = np.append(ts, selection[2])
    z_observations = np.append(z_observations, next_observation)
